# Create a Basic Convolutional Neural Network for Image Classification

## 1. Import Library and Data

In [1]:
#@title Code for loading the data
# import libraries 
import tensorflow as tf 
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

# utilities library
import cv2
import os

import random 
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)
os.environ['PYTHONHASHSEED'] = '0'


# !rm -rf jaffe
# !rm -rf __MACOSX
# !rm jaffe.zip
!wget https://github.com/iqbalbasyar/kickstart_gan/raw/main/jaffe.zip -q
!unzip -q jaffe.zip 


data_path = 'jaffe'
data_dir_list = os.listdir(data_path)

img_data_list=[]
img_label = []
label_map = {}
i = 0 

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    label_map[i] = str(dataset)
    
    # print ('Load the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        img_label.append(i)
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)
    i += 1

img_data = np.array(img_data_list)
img_data = img_data.astype('float')
img_data = img_data/255 #normalization
img_label = np.array(img_label)

# num_classes = 6
# num_of_samples = img_data.shape[0]
# img_label = np.ones((num_of_samples,),dtype='int')

# img_label[0:29]=0 #30
# img_label[30:58]=1 #29
# img_label[59:90]=2 #32
# img_label[91:121]=3 #31
# img_label[122:152]=4 #31
# img_label[153:]=5 #30
# names = ['ANGRY','DISGUST','FEAR','HAPPY','SAD','SURPRISE']

def getlabel(id):
    return label_map[id]

**About this Data**: 

We will use the Japanese Female Facial Expression (JAFFE) dataset which has 183 images of 10 different female models posing for 6 emotions. The data will be normalized so its value will ranging from 0 to 1. 

The label will be denoted as numbers, which follow this mapping : 

|    | Expression   |
|---:|:-------------|
|  0 | HAPPY         |
|  1 | SURPRISE          |
|  2 | FEAR        |
|  3 | SAD     |
|  4 | ANGRY        |
|  5 | DISGUST      |

<details> <summary>Cite the author</summary>
Michael J. Lyons, Shigeru Akamatsu, Miyuki Kamachi, Jiro Gyoba. Coding Facial Expressions with Gabor Wavelets, 3rd IEEE International Conference on Automatic Face and Gesture Recognition, pp. 200-205 (1998). http://doi.org/10.1109/AFGR.1998.670949 Open access content available at: https://zenodo.org/record/3430156

</details>

split the data into train and test set so that we can perform **cross validation** later

In [2]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(img_data, img_label, test_size=0.1, random_state=0)

___

## 2. Create the Model

In previous two notebook, we already created a similar model using keras. Here's the code if you wanted to remember.

In [3]:
# simple model in 1st Notebook
model_simple = keras.Sequential()
model_simple.add(keras.layers.Dense(units=1, input_shape=[1]))

model_simple.compile(optimizer='SGD', loss='mean_squared_error')

In [4]:
# Neural Network in 2nd Notebook
model_nn = keras.Sequential()

model_nn.add(keras.layers.Flatten(input_shape=(128,128,3)) ) # flatten layer 
model_nn.add(keras.layers.Dense(units = 128,activation='relu') )  #input layer 
model_nn.add(keras.layers.Dense(units = 64, activation='relu') )  # hidden layer 
model_nn.add(keras.layers.Dense(units = 6, activation='softmax') ) # output layer 

model_nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

We are now trying to add a convolutional part to our model. Mind the changes compared to previous basic neural network

In [5]:
model_cnn = keras.Sequential()

model_cnn.add(keras.layers.Conv2D(filters=16, kernel_size=(5,5), input_shape=(128,128,3), padding='same', activation="relu"))
model_cnn.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation="relu"))
model_cnn.add(keras.layers.MaxPooling2D())

model_cnn.add(keras.layers.Flatten()) # flatten layer 
model_cnn.add(keras.layers.Dense(units = 128,activation='relu') )  #input layer 
model_cnn.add(keras.layers.Dense(units = 64, activation='relu') )  # hidden layer 
model_cnn.add(keras.layers.Dense(units = 6, activation='softmax') ) # output layer 

model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
# summarize the model to see what happened inside
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      1216      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16777344  
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

## 3. Train the Model

In [7]:
model_cnn.fit(x_train, y_train, epochs=20);

Epoch 1/20
6/6 [==============================] - 0s 55ms/step - loss: 7.0799 - accuracy: 0.1890
Epoch 2/20
6/6 [==============================] - 0s 33ms/step - loss: 1.9150 - accuracy: 0.2073
Epoch 3/20
6/6 [==============================] - 0s 33ms/step - loss: 1.8054 - accuracy: 0.1829
Epoch 4/20
6/6 [==============================] - 0s 33ms/step - loss: 1.7506 - accuracy: 0.2256
Epoch 5/20
6/6 [==============================] - 0s 32ms/step - loss: 1.7592 - accuracy: 0.2622
Epoch 6/20
6/6 [==============================] - 0s 33ms/step - loss: 1.6657 - accuracy: 0.3354
Epoch 7/20
6/6 [==============================] - 0s 32ms/step - loss: 1.5612 - accuracy: 0.4817
Epoch 8/20
6/6 [==============================] - 0s 33ms/step - loss: 1.5319 - accuracy: 0.3476
Epoch 9/20
6/6 [==============================] - 0s 32ms/step - loss: 1.4789 - accuracy: 0.3963
Epoch 10/20
6/6 [==============================] - 0s 31ms/step - loss: 1.3574 - accuracy: 0.4146
Epoch 11/20
6/6 [============

## 4. Test the Model

In [8]:
score_accuracy = model_cnn.evaluate(x_test, y_test)

1/1 [==============================] - 0s 2ms/step - loss: 1.0605 - accuracy: 0.8421


# Comparing Neural Network with CNN

In [12]:
# train the basic neural network we previously have defined
model_nn.fit(x_train, y_train, epochs=40)

Epoch 1/40
6/6 [==============================] - 0s 7ms/step - loss: 0.4387 - accuracy: 0.8720
Epoch 2/40
6/6 [==============================] - 0s 7ms/step - loss: 0.3853 - accuracy: 0.9085
Epoch 3/40
6/6 [==============================] - 0s 8ms/step - loss: 0.3910 - accuracy: 0.9329
Epoch 4/40
6/6 [==============================] - 0s 7ms/step - loss: 0.3564 - accuracy: 0.9451
Epoch 5/40
6/6 [==============================] - 0s 8ms/step - loss: 0.4420 - accuracy: 0.8720
Epoch 6/40
6/6 [==============================] - 0s 7ms/step - loss: 0.6268 - accuracy: 0.7805
Epoch 7/40
6/6 [==============================] - 0s 8ms/step - loss: 0.5315 - accuracy: 0.8293
Epoch 8/40
6/6 [==============================] - 0s 7ms/step - loss: 0.3420 - accuracy: 0.9024
Epoch 9/40
6/6 [==============================] - 0s 7ms/step - loss: 0.4113 - accuracy: 0.8902
Epoch 10/40
6/6 [==============================] - 0s 7ms/step - loss: 0.3709 - accuracy: 0.8841
Epoch 11/40
6/6 [======================

In [13]:
score_accuracy = model_nn.evaluate(x_test, y_test)

1/1 [==============================] - 0s 2ms/step - loss: 1.2335 - accuracy: 0.6842
